In [1]:
import pygame as game
from pygame.locals import *
from collections import deque
import random
import os
os.chdir("/home/agastya/flappy-bird")

In [2]:
bird_sprite = game.image.load("bird.png")
g = 0.0017 #gravity
bird_height = 80
bird_width = 80
bird_velocity = 0.1
init_x = 10
init_y = 200
pipe_width = 100
pipe_piece_height = 15
pipe_add_rate = 750
pipe_scroll_speed = 0.5
screen_width = 500
screen_height = 600
game.init()
screen = game.display.set_mode((screen_width, screen_height))
myfont = game.font.SysFont("monospace", 32)

In [3]:
class Bird(game.sprite.Sprite):
    """Initialization and update of motion parameters"""
    
    def __init__(self):
        game.sprite.Sprite.__init__(self)
        self.x = init_x
        self.y = init_y
        
        #fitting bird.png in bounding box
        self.sprite = game.transform.scale(bird_sprite, (bird_height, bird_width))
        self.velocity = bird_velocity
        self.mask = game.mask.from_surface(self.sprite)
        
    def update_no_flap(self):
        self.velocity += g
        self.y += self.velocity
    
    def update_flap(self):
        self.velocity -= bird_velocity*10
        if self.velocity < -bird_velocity*5:
            self.velocity -= -bird_velocity*5
    
    #new copy of bounding box of bird
    @property
    def rect(self):
        return Rect(self.x, self.y, bird_height, bird_width)

In [4]:
class Pipe(game.sprite.Sprite):
    
    def __init__(self):
        self.x = screen_width - 2    #position of pipe at init
        self.passed_flag = False    #flag that checks whether bird has crossed pipe or not
        
        self.sprite = game.Surface((pipe_width, screen_height), SRCALPHA)
        self.sprite.convert()
        self.sprite.fill((0,0,0,0))
        
        # Set the length of the total vertical length covered by the pipe pair on the screen
        total_pipe_body_pieces = int((screen_height - 3*bird_height - 3*pipe_piece_height)/pipe_piece_height)
        
        # Set the vertical length of the bottom piece of the pipe pair
        self.bottom_pieces = random.randint(1, total_pipe_body_pieces - 1)
        
        # Set the vertical length of the to piece of the pipe pair
        self.top_pieces = total_pipe_body_pieces - self.bottom_pieces
        
        #-> be me
        #-> pipe.png
        pipe_sprite = game.image.load("pipe.png")
        pipe_sprite = game.transform.scale(pipe_sprite, (pipe_width, 100))
        
        for i in range(1, self.bottom_pieces + 1):
            piece_pos = (0, screen_height - i*pipe_piece_height)
            self.sprite.blit(pipe_sprite, piece_pos)
            
        for i in range(self.top_pieces):
            self.sprite.blit(pipe_sprite, (0, i*pipe_piece_height))
            
        self.top_pieces += 1
        self.bottom_pieces += 1
        self.bot_height = self.bottom_pieces*pipe_piece_height
        
        #collision detection
        self.mask = game.mask.from_surface(self.sprite)
    
    #new bounding box pipe
    @property
    def rect(self):
        return Rect(self.x, 0, pipe_width, pipe_piece_height)
    #pipe on screen?
    @property
    def visible(self):
        return -pipe_width<self.x<screen_width
    #update x-coordinates
    def update(self):
        self.x -= pipe_scroll_speed
    #bird collides with pipe?
    def collides_with(self, bird):
        return game.sprite.collide_mask(self, bird)

In [5]:
def play(headless=False, ai=None):
    
    bird = Bird()
    pipes = deque()
    frame=0
    score=0
    done=False
    
    while True:
        #draw bird on screen
        bird.update_no_flap()
        if not headless:
            screen.fill((0, 255, 255))
            screen.blit(bird.sprite, (bird.x, bird.y))
        if frame%pipe_add_rate == 0:
            p = Pipe()
            pipes.append(p)
        #collision?
        collision_detected = any(p.collides_with(bird) for p in pipes)
        if bird.y > 600 or bird.y < 0 or collision_detected:
            break
        while pipes and not pipes[0].visible:
            pipes.popleft()
        #Bird crosses pipe
        for p in pipes:
            if p.x + pipe_width/2 <  bird.x and not p.passed_flag:
                p.passed_flag += True    #score increased by 1
                score += 1
            p.update()
            
            if not headless:
                screen.blit(p.sprite, (p.x, 0))
                
        if not ai:
            for event in game.event.get():
                if event.type == game.QUIT:
                    break
                if event.type == game.KEYDOWN and event.key == game.K_SPACE:
                    bird.update_flap()
        elif ai.forward([bird.x - pipes[0].x, screen_height - bird.y - pipes[0].bot_height]) > 0.5:
            bird.update_flap()
        label = myfont.render("Score = " + str(score), 1, (255, 100, 0))
        screen.blit(label, (316, 40))
        
        if not headless:
            game.display.flip()
        frame += 1
    print('Score: ' + str(score))
    return score

In [18]:
play()

Score: 1


1